In [1]:
import MySQLdb
import math
import itertools
import numpy as np
import pickle
from statsmodels import robust
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn import neighbors
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def generateFeatures(Temp):
    f0=np.mean(Temp[:,0])
    f1=np.mean(Temp[:,1])
    f2=np.mean(Temp[:,2])
    f3=np.std(Temp[:,0])
    f4=np.std(Temp[:,1])
    f5=np.std(Temp[:,2])
    f6=np.var(Temp[:,0])
    f7=np.var(Temp[:,1])
    f8=np.var(Temp[:,2])
    f9=robust.mad(Temp[:,0])
    f10=robust.mad(Temp[:,1])
    f11=robust.mad(Temp[:,2])
    f12=abs(np.mean(Temp[:,0])-np.mean(Temp[:,1]))
    f13=abs(np.mean(Temp[:,0])-np.mean(Temp[:,2]))
    f14=abs(np.mean(Temp[:,1])-np.mean(Temp[:,2]))
    return [f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14]

In [ ]:
try:
    db = MySQLdb.connect(host="localhost",user="root",passwd="pass",db="human_activity_recognition") 
except:
    print "I am unable to connect to the database"

In [ ]:
pickle_in=open('RF.pickle','rb')
clf=pickle.load(pickle_in)

In [ ]:
cur = db.cursor()
curIn = db.cursor()

In [ ]:
cur.execute("SELECT * FROM testing_data WHERE label is NULL;")

In [ ]:
rows = cur.fetchall()
testingData=np.empty((0,4),float)
#print "\nShow me the databases:\n"
for row in rows:
    print "   ",row[0], row[1], row[2], row[3], row[4]
    testingData=np.vstack((testingData, [row[1], row[2], row[3], row[0]] ))
    if len(testingData)==25:
        pre=clf.predict(generateFeatures(testingData[:,0:3]))
        
        mylist = [(int(pre),i) for i in range(int(testingData[0,3]),int(testingData[-1,3])+1)]
        #print(mylist)
        print(pre)
        
        curIn.executemany('''UPDATE testing_data SET label = (%s) WHERE id = (%s);''', mylist)
        testingData=np.empty((0,4),float)
        
    
    

In [ ]:
db.commit()
db.close()